

---

# **Install Depedencies**

In [1]:
!pip install yfinance xgboost scikit-learn pandas numpy matplotlib plotly streamlit joblib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 114.5 MB/s eta 0:00:00


# **Main App.py**

In [15]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import plotly.express as px

# ---------- Helper functions ----------
def fetch_yfinance(ticker="AAPL", period="5y"):
    df = yf.download(ticker, period=period, interval="1d", progress=False)
    df = df[["Open","High","Low","Close","Volume"]]
    df.index = pd.to_datetime(df.index)
    return df

def sma(series, window): return series.rolling(window).mean()
def ema(series, span): return series.ewm(span=span, adjust=False).mean()

def rsi(series, period=14):
    delta = series.diff()
    up = delta.clip(lower=0)
    down = -1 * delta.clip(upper=0)
    ma_up = up.rolling(period).mean()
    ma_down = down.rolling(period).mean()
    rs = ma_up / (ma_down + 1e-9)
    return 100 - (100 / (1 + rs))

def macd(series, fast=12, slow=26, signal=9):
    ema_fast = ema(series, fast)
    ema_slow = ema(series, slow)
    macd_line = ema_fast - ema_slow
    signal_line = macd_line.ewm(span=signal, adjust=False).mean()
    hist = macd_line - signal_line
    return macd_line, signal_line, hist

def bollinger_bands(series, window=20, n_std=2):
    sma_ = series.rolling(window).mean()
    std_ = series.rolling(window).std()
    upper = sma_ + n_std*std_
    lower = sma_ - n_std*std_
    return upper, lower

def build_features(df):
    df = df.copy()
    df["Return"] = df["Close"].pct_change()
    for w in [7,14,21]:
        df[f"SMA_{w}"] = sma(df["Close"], w)
    df["RSI_14"] = rsi(df["Close"], 14)
    macd_line, macd_signal, macd_hist = macd(df["Close"])
    df["MACD_line"], df["MACD_signal"], df["MACD_hist"] = macd_line, macd_signal, macd_hist
    upper, lower = bollinger_bands(df["Close"])
    df["BB_upper"], df["BB_lower"] = upper, lower
    df["Volume_change"] = df["Volume"].pct_change()
    df["Target"] = (df["Close"].shift(-1) > df["Close"]).astype(int)
    return df.dropna()

def train_model(df):
    feature_cols = [c for c in df.columns if c not in ["Target","Open","High","Low"]]
    split = int(len(df)*0.8)
    train, test = df.iloc[:split], df.iloc[split:]
    X_train, y_train = train[feature_cols], train["Target"]
    X_test, y_test = test[feature_cols], test["Target"]

    scaler = StandardScaler()
    X_train_s = scaler.fit_transform(X_train)
    X_test_s = scaler.transform(X_test)

    model = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
    model.fit(X_train_s, y_train)
    preds = model.predict(X_test_s)
    proba = model.predict_proba(X_test_s)[:,1]

    metrics = {
        "Accuracy": accuracy_score(y_test,preds),
        "Precision": precision_score(y_test,preds),
        "Recall": recall_score(y_test,preds),
        "F1": f1_score(y_test,preds),
        "ROC_AUC": roc_auc_score(y_test,proba)
    }
    return model, scaler, feature_cols, metrics, test, preds, proba

# ---------- Streamlit UI ----------
st.set_page_config(page_title="Stock Trend Predictor", layout="wide")
st.title("💹 ML-Powered Stock Trend Predictor")

ticker = st.text_input("Enter Stock Ticker (yfinance symbol)", "AAPL").upper()
period = st.selectbox("Select Period", ["3y","5y","10y"], index=1)

if st.button("Fetch & Train Model"):
    with st.spinner("Fetching data..."):
        df = fetch_yfinance(ticker, period)
        df_feat = build_features(df)
    st.success(f"Fetched {len(df)} rows and built {len(df_feat.columns)} features.")

    with st.spinner("Training model..."):
        model, scaler, feats, metrics, test_df, preds, proba = train_model(df_feat)
    st.success("Model trained successfully!")
    st.write("### Model Performance Metrics")
    st.json(metrics)

    # ---------- Price Chart ----------
    fig1 = px.line(
        x=df_feat.index,
        y=df_feat["Close"].values.ravel(),  # ensure 1D
        title=f"{ticker} Close Price ({period})"
    )
    st.plotly_chart(fig1, use_container_width=True)

    # ---------- Predicted Trend Chart ----------
    test_df = test_df.copy()
    test_df["Pred"] = preds
    test_df["Prob"] = proba
    fig2 = px.line(
        x=test_df.index,
        y=test_df["Close"].values.ravel(),  # ensure 1D
        color=test_df["Pred"].map({0:"Down",1:"Up"}),
        title="Predicted Trend (Test Period)"
    )
    st.plotly_chart(fig2, use_container_width=True)

    # ---------- Cumulative Returns ----------
    test_df["Strategy_return"] = test_df["Return"] * test_df["Pred"]
    test_df["Strategy_cum"] = (1+test_df["Strategy_return"]).cumprod()
    test_df["Buy_hold_cum"] = (1+test_df["Return"]).cumprod()
    fig3 = px.line(
        test_df,
        x=test_df.index,
        y=[test_df["Strategy_cum"].values.ravel(), test_df["Buy_hold_cum"].values.ravel()],
        labels={"y":"Cumulative Returns"},
        title="Cumulative Returns (Strategy vs Buy & Hold)"
    )
    st.plotly_chart(fig3, use_container_width=True)

    # ---------- Next-Day Prediction ----------
    latest = df_feat.iloc[[-1]]
    X_latest = scaler.transform(latest[feats])
    pred = model.predict(X_latest)[0]
    prob = model.predict_proba(X_latest)[0,1]
    label = "📈 UP" if pred==1 else "📉 DOWN"
    st.subheader("Next-Day Prediction")
    st.write(f"Prediction: {label}  (Prob of UP = {prob:.2f})")


Writing app.py


In [2]:
!pip install pyngrok

In [4]:
!pip install streamlit

In [5]:
!ngrok config add-authtoken 2y1h90IDQ09WkydpMkH0V1uwDoY_48ETKKBgkK1u6cGDHcwm2

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [16]:
from pyngrok import ngrok
import subprocess
import threading
import time

ngrok.kill()
def run_streamlit():
    subprocess.call(["streamlit", "run", "app.py", "--server.port=8503",
                     "--server.headless=true", "--server.enableCORS=false"])
thread = threading.Thread(target=run_streamlit)
thread.start()
time.sleep(5)

public_url = ngrok.connect(8503)
print(f"Streamlit Application Link: {public_url}")

Streamlit Application Link: NgrokTunnel: "https://b68e1474428b.ngrok-free.app" -> "http://localhost:8503"


In [13]:
ngrok.kill()